In [1]:
import pandas as pd
from rutas import CSV_BIBLIOTECAS, CSV_CINES, CSV_MUSEOS


### 2 Procesamiento de Datos
<span>Normalizar toda la información de Museos, Salas de Cine y Bibliotecas Populares, para crear una única tabla que contenga:</span>

<style>
    ul{columns:4;}
    div{margin-top:25px;}
</style>
<div class='lista'>
    <ul>
        <li>cod_localidad</li>
        <li>id_provincia</li>
        <li>id_departamento</li>
        <li>categoría</li>
        <li>provincia</li>
        <li>localidad</li>
        <li>nombre</li>
        <li>domicilio</li>
        <li>código postal</li>
        <li>número de teléfono</li>
        <li>mail</li>
        <li>web</li>
    </ul>
</div>


#### 2.1 Normalizando y consolidando campos

In [33]:
# Leer csvs

raw_df_bibliotecas = pd.read_csv(CSV_BIBLIOTECAS)
raw_df_museos = pd.read_csv(CSV_MUSEOS)
raw_df_cines = pd.read_csv(CSV_CINES)

In [21]:
raw_df_bibliotecas.columns = raw_df_bibliotecas.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')


In [32]:
raw_df_bibliotecas.Categoria.unique()

array(['Bibliotecas Populares'], dtype=object)

In [34]:
def normalizar_campos(csv_file):
    # Leer csv
    raw_df = pd.read_csv(csv_file)
    # Normalizar campos
    raw_df.columns = raw_df.columns.str.lower()
    raw_df.columns = raw_df.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
    return raw_df

raw_df_bibliotecas = normalizar_campos(CSV_BIBLIOTECAS)
raw_df_museos = normalizar_campos(CSV_MUSEOS)
raw_df_cines = normalizar_campos(CSV_CINES)

In [38]:
def nuevos_campos(df:pd.DataFrame):
    match df.categoria[0]:
        case 'Salas de cine':
            df[['número de teléfono', 'mail']] = 's/d'
            new_df = df[['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'número de teléfono','mail' , 'web']]
        
        case 'Espacios de Exhibición Patrimonial':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'telefono', 'mail', 'web']]
        
        case 'Bibliotecas Populares':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'domicilio', 'cp', 'telefono', 'mail', 'web']]
    
    return new_df

# Seleccionar columnas requeridas en cada dataframe
df_bibliotecas = nuevos_campos(raw_df_bibliotecas)
df_museos = nuevos_campos(raw_df_museos)
df_cines = nuevos_campos(raw_df_cines)


In [3]:
def set_columns_lower(df):
    '''
    Convierte los nombres de las variables
    del dataframe a minúsculas
    '''
    df.columns = df.columns.str.lower()


In [5]:
set_columns_lower(raw_df_bibliotecas)
set_columns_lower(raw_df_museos)
set_columns_lower(raw_df_cines)

In [8]:
# Crear lista de nomenclaturas para nueva tabla
columnas = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoría', 
            'provincia', 'localidad', 'nombre', 'domicilio', 'código postal', 
            'número de teléfono', 'mail', 'web']

In [9]:
# Seleccionar columnas necesarias en cada dataframe
df_bibliotecas = raw_df_bibliotecas[['cod_loc', 'idprovincia', 'iddepartamento', 'categoría', 'provincia', 'localidad', 'nombre', 'domicilio', 'cp', 'teléfono', 'mail', 'web']]
df_museos = raw_df_museos[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'telefono', 'mail', 'web']]

# Cines necesita columnas para telefono y mail
raw_df_cines[['número de teléfono', 'mail']] = 's/d'
df_cines = raw_df_cines[['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'número de teléfono','mail' , 'web']]

In [10]:
# Renombrando columnas
def rename_cols(df, new_names):
    '''
    Crea un diccionario a partir de un zip entre las columnas actuales 
    del dataframe y una lista con sus correspondientes nuevos nombres
    Pasar como primer argumento el dataframe con las columnas a renombrar
    Pasar como segundo argumento la lista con los nuevos nombres de las columnas
    Los indices de ambos argumentos deben coincidir para que se descompaginen los nombres
    '''
    diccionario = dict(zip(df.columns, new_names))
    df = df.rename(columns=diccionario)
    return df


In [11]:
# Crear dataframes con columnas renombradas
df_bibliotecas = rename_cols(df_bibliotecas, columnas)
df_museos = rename_cols(df_museos, columnas)
df_cines = rename_cols(df_cines, columnas)

#### 2.2 Normalizando datos

In [12]:
df_bibliotecas.fillna('s/d', inplace=True)
df_museos.fillna('s/d', inplace=True)
df_cines.fillna('s/d', inplace=True)

#### 2.3 Crear tabla única

In [13]:
consolidado = pd.concat([df_bibliotecas,df_museos,df_cines], ignore_index=True)

### 3 Procesando los datos conjuntos


#### 3.1 Generar una tabla con la siguiente información:
- Cantidad de registros totales por categoría
- Cantidad de registros totales por fuente
- Cantidad de registros por provincia y categoría

In [44]:
registros_categoria = pd.DataFrame(consolidado.groupby(by= 'categoría')['categoría'].count())
registros_categoria.columns=['total_registros']

In [45]:
registros_categoria

,total_registros
categoría,
Bibliotecas Populares,2017
Espacios de Exhibición Patrimonial,1182
Salas de cine,316


In [46]:
registros_fuente = pd.concat([raw_df_bibliotecas,raw_df_museos,raw_df_cines], ignore_index=True)
registros_fuente = pd.DataFrame(registros_fuente.groupby(by='fuente')['fuente'].count())
registros_fuente.columns=['total_registros']

In [47]:
registros_fuente

,total_registros
fuente,
CNMLH - Enlace SInCA,1
CNMLH - Ente Cultural de Tucumán,1
CONABIP,2010
DNPyM,1048
Dirección Provincial de Patrimonio Cultural - Enlace SInCA,1
Dirección de Cultura de Villa Allende,2
Dirección de Cultura y de Educación de la Municipalidad de Unquillo,1
Gob. Pcia.,1
Gobierno de la Provincia,14


In [48]:
registros_provincia_categoria = pd.DataFrame(consolidado.groupby(by=['provincia', 'categoría'])['provincia'].count())
registros_provincia_categoria.columns = ['total_registros']

In [49]:
registros_provincia_categoria

total_registros
provincia                                          categoría                                          
Buenos Aires                                       Bibliotecas Populares                           543
                                                   Espacios de Exhibición Patrimonial              152
                                                   Salas de cine                                   101
Catamarca                                          Bibliotecas Populares                            38
                                                   Espacios de Exhibición Patrimonial               32
...                                                                                                ...
Tierra del Fuego                                   Salas de cine                                     3
Tierra del Fuego, Antártida e Islas del Atlánti... Espacios de Exhibición Patrimonial               14
Tucumán                                            Bibliotecas Populares                            37
                                                   Espacios de Exhibición Patrimonial               26
                                                   Salas de cine                                    11

[73 rows x 1 columns]

#### 3.2 Procesar la información de cines
Crear una tabla que contenga:
- Provincia
- Cantidad de pantallas
- Cantidad de butacas
- Cantidad de espacios INCAA

In [137]:
tabla_cines = raw_df_cines[['provincia', 'pantallas', 'butacas', 'espacio_incaa']].copy()
tabla_cines['espacio_incaa'] = (tabla_cines['espacio_incaa'] == 'Si').astype(int)
tabla_cines = tabla_cines.groupby(by='provincia').sum()